## Exercise Set 3 for OSM 

### Dynamic Programming with John Stachurski

### Solutions by Wei Han Chia

Exercises for the [OSM](https://bfi.uchicago.edu/osm) bootcamp dynamic programming section.

We will use the following libraries:

In [4]:
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt

### Exercise 1.

Using Numba, as discussed in [this lecture](https://lectures.quantecon.org/py/need_for_speed.html), write your own version of NumPy's [interp](https://docs.scipy.org/doc/numpy/reference/generated/numpy.interp.html) function, specializing in linear interpolation in one dimension.  

Note that NumPy's function is compiled native machine code and hence is fast.  But try to beat if you can, at least in some scenarios, by using Numba to speed up your code.  Show a time comparison between the two functions, for some suitable choice of test.

In [146]:
from numba import jit
import timeit

def lininterp(x, xp, yp):
    y = np.ones_like(x)
    xlen = len(x)
    length = len(yp)
    for j in range(0,xlen):
        xtemp = x[j]
        if xtemp < xp[0]:
            y[j] = yp[0]
        elif xtemp > xp[length-1]:
            y[j] = yp[length-1]
        else:
            for i in range(0,length-2):
                if xtemp > xp[i] and xtemp < xp[i+1]:
                    y[j] = (yp[i]*(xp[i+1] - xtemp) + yp[i+1]*(xtemp - xp[i]))/(xp[i+1] - xp[i])
    return y

# Lets test interpolation speed on a quadratic function
xp_test = np.linspace(2,7,1000)
yp_test = xp_test ** 2
x_test = np.linspace(3.5,5,100)

lininterp_numba = jit(lininterp)

In [149]:
start_time = timeit.default_timer()
lininterp(x_test,xp_test,yp_test)
print("Time without JIT: ", timeit.default_timer() - start_time)

start_time_jit = timeit.default_timer()
lininterp_numba(x_test,xp_test,yp_test)
print("Time with JIT: ", timeit.default_timer() - start_time_jit)

start_time_np = timeit.default_timer()
np.interp(x_test,xp_test,yp_test)
print("Time with np.interp: ", timeit.default_timer() - start_time_np)

Time without JIT:  0.22047310671769083
Time with JIT:  0.000943280931096524
Time with np.interp:  0.000750226026866585


With numba, the timing of our homemade linear interpolation function approaches the same speed as that of np.interp. This is a significant speedup from the time without numba.

### Exercise 2

Using your "Numbafied" linear interpolation function, try to use Numba to additionally speed up the endogenous grid method code from [this lecture](https://lectures.quantecon.org/py/egm_policy_iter.html).  Use CRRA utility and Cobb-Douglas production, as in that lecture, with the following parameter values.


Note: I didn't get much speed up.  I think because the outer loops don't matter much for speed, and hence it doesn't gain us much when we compile them.  

See how you go.

In [138]:
import quantecon as qe

## Define the model

alpha = 0.65
beta = 0.95
mu = 0
s = 0.1
grid_min = 1e-6
grid_max = 4
grid_size = 200
shock_size = 250

gamma = 1.5   # Preference parameter
gamma_inv = 1 / gamma

def f(k):
    return k**alpha

def f_prime(k):
    return alpha * k**(alpha - 1)

def u(c):
    return (c**(1 - gamma) - 1) / (1 - gamma)

def u_prime(c):
    return c**(-gamma)

def u_prime_inv(c):
    return c**(-gamma_inv)

k_grid = np.linspace(grid_min, grid_max, grid_size)
shocks = np.exp(mu + s * np.random.randn(shock_size))

import numpy as np

def coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
    """
    The approximate Coleman operator, updated using the endogenous grid
    method.  
    
    Parameters
    ----------
    g : function
        The current guess of the policy function
    k_grid : array_like(float, ndim=1)
        The set of *exogenous* grid points, for capital k = y - c
    beta : scalar
        The discount factor
    u_prime : function
        The derivative u'(c) of the utility function
    u_prime_inv : function
        The inverse of u' (which exists by assumption)
    f : function
        The production function f(k)
    f_prime : function
        The derivative f'(k)
    shocks : numpy array
        An array of draws from the shock, for Monte Carlo integration (to
        compute expectations).

    """

    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    
    # Determine endogenous grid
    y = k_grid + c  # y_i = k_i + c_i

    # Update policy function and return
    Kg = lambda x: lininterp_numba(x, y, c)
    return Kg

def crra_coleman_egm(g):
    return coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

sim_length = 20

crra_coleman_egm_numba = jit(crra_coleman_egm)


In [141]:
print("Timing policy function iteration with endogenous grid")
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_egm_numba(g)
    g = new_g
qe.util.toc()

Timing policy function iteration with endogenous grid
TOC: Elapsed: 0.47135186195373535 seconds.


0.47135186195373535

It doesn't seem like numba speeds up the implementation for the egp method significantly compared to the results presented in the lecture notes. In fact it would appear that our inefficient linear interpolation function slows down the overall speed of the endogenous grid method slightly.